#Fingerprint

### [데이터세트 구축]

#0801 FTO 학습 및 테스트 시작
* potency 1 (235건) potency 0 (118건) 총 535건 
* Potency 0은 실험데이터이므로 전부 들어가도록하고, 모자라는 부분만 decoy 랜덤으로 select 해서 총 LABEL0:LABEL1 = 1: 1,3,5,10,20,50 데이터 세트 구축함. 

#### 1. CPI 데이터 수집

1. ChemBL, Binding DB (Alpha-ketoglutarate-dependent dioxygenase FTO, Uniprot ID Q9C0B1) 에서 CPI 데이터 다운로드 후 DB구축 -> 1차 중복제거
2. Canonical smiles로 변환 후 2차 중복제거
3. 결과; potency 1 (235건) potency 0 (118건) 총 535건 

In [9]:
#Canonical smiles변환

#필요 패키지 불러오기
from rdkit import Chem
import pandas as pd

#데이터 불러오기
dataset = pd.read_csv(r"C:\FTO_data\FTO_data.csv")
print("Data shape:", dataset.shape)
print(dataset.head(), "\n")

Data shape: (488, 2)
                                       Ligand SMILES  potency
0             c1ccc(-c2ccc(CNCC3(N4CCCC4)COC3)o2)cc1        1
1             C(NCC1(COC1)N1CCCC1)c1ccc(o1)-c1ccccc1        1
2              C(NCC1(COC1)N1CCCC1)C1CCC(O1)c1ccccc1        1
3  Cc1c(Nc2ccncc2C(=O)NNC(=O)/C=C/C(=O)O)cc(Cl)c(...        1
4  Cc1c(Nc2ccncc2C(=O)NNC(=O)\C=C\C(O)=O)cc(Cl)c(...        1 



In [10]:

#원본 SMILES 컬럼명 바꾸기
dataset = dataset.rename(columns={'Ligand SMILES': 'raw_SMILES'})
print("컬럼명 변경 후:", dataset.columns.tolist(), "\n")

#Canonical smiles 생성 
def canonical_smiles(smiles_list, include_stereo=True): #include_stereo: True면 입체화학(@,@@,/,\\) 정보 보존
    canonical = [] #결과를 담을 빈 리스트
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        #Mol이 None이 아니면 Moltosmiles, 아니면 None
        canonical.append(Chem.MolToSmiles(mol, isomericSmiles=include_stereo) if mol else None)
    return canonical

#canonical_SMILES컬럼을 추가하여 데이터 저장
Canon_SMILES = canonical_smiles(dataset['raw_SMILES']) #리스트 생성
assert len(Canon_SMILES) == len(dataset), "길이가 맞지 않습니다!" #원본 행 개수와 동일한지 검증 
dataset.insert(0, 'canonical_SMILES', Canon_SMILES)
print("데이터 shape (행, 열):", dataset.shape)
print(dataset[['canonical_SMILES', 'raw_SMILES']].head())

컬럼명 변경 후: ['raw_SMILES', 'potency'] 

데이터 shape (행, 열): (488, 3)
                                    canonical_SMILES  \
0             c1ccc(-c2ccc(CNCC3(N4CCCC4)COC3)o2)cc1   
1             c1ccc(-c2ccc(CNCC3(N4CCCC4)COC3)o2)cc1   
2              c1ccc(C2CCC(CNCC3(N4CCCC4)COC3)O2)cc1   
3  Cc1c(Nc2ccncc2C(=O)NNC(=O)/C=C/C(=O)O)cc(Cl)c(...   
4  Cc1c(Nc2ccncc2C(=O)NNC(=O)/C=C/C(=O)O)cc(Cl)c(...   

                                          raw_SMILES  
0             c1ccc(-c2ccc(CNCC3(N4CCCC4)COC3)o2)cc1  
1             C(NCC1(COC1)N1CCCC1)c1ccc(o1)-c1ccccc1  
2              C(NCC1(COC1)N1CCCC1)C1CCC(O1)c1ccccc1  
3  Cc1c(Nc2ccncc2C(=O)NNC(=O)/C=C/C(=O)O)cc(Cl)c(...  
4  Cc1c(Nc2ccncc2C(=O)NNC(=O)\C=C\C(O)=O)cc(Cl)c(...  


In [11]:

#Create a list for duplicate smiles
duplicates_smiles = dataset[dataset['canonical_SMILES'].duplicated()]['canonical_SMILES'].values
print("제거될 중복 개수:", len(duplicates_smiles))

# 중복된 행 확인 (정렬)
dup_rows = dataset[dataset['canonical_SMILES'].isin(duplicates_smiles)].sort_values(by=['canonical_SMILES'])
print("중복된 샘플들:")
print(dup_rows[['canonical_SMILES', 'raw_SMILES']].head(), "\n")

#Drop duplicate values
dataset_unique = dataset.drop_duplicates(subset=['canonical_SMILES'])
print("중복 제거 후 남은 행 수:", len(dataset_unique))
print(dataset_unique.head())

dataset_unique.to_csv(r"C:\FTO_data\FTO_data_cannon.csv", index=False)

제거될 중복 개수: 135
중복된 샘플들:
                                      canonical_SMILES  \
426                                       CC(=O)C(=O)O   
435                                       CC(=O)C(=O)O   
437  CC(C)(COP(=O)(O)OP(=O)(O)OC[C@H]1O[C@@H](n2cnc...   
422  CC(C)(COP(=O)(O)OP(=O)(O)OC[C@H]1O[C@@H](n2cnc...   
393       CC(C)[C@@H](NC(=O)c1nc(Cl)c2ccccc2c1O)C(=O)O   

                                            raw_SMILES  
426                                       CC(=O)C(=O)O  
435                                       CC(=O)C(O)=O  
437  CC(C)(COP(O)(=O)OP(O)(=O)OC[C@H]1O[C@H]([C@H](...  
422  CC(C)(COP(=O)(O)OP(=O)(O)OC[C@H]1O[C@@H](n2cnc...  
393       CC(C)[C@@H](NC(=O)c1nc(Cl)c2ccccc2c1O)C(=O)O   

중복 제거 후 남은 행 수: 353
                                    canonical_SMILES  \
0             c1ccc(-c2ccc(CNCC3(N4CCCC4)COC3)o2)cc1   
2              c1ccc(C2CCC(CNCC3(N4CCCC4)COC3)O2)cc1   
3  Cc1c(Nc2ccncc2C(=O)NNC(=O)/C=C/C(=O)O)cc(Cl)c(...   
5              Fc1ccc2[nH]cc(CNCC3(N4CC

#### 2. DUDE 데이터 수집

1. DUDE생성 https://dude.docking.org/ 이용. (생성에 오류떠서 안씀; https://tldr.docking.org/)
2. 파일 합치기
3. DUDE cannon smiles로 변경 -> 중복제거 (코드) -> 중복제거 1회 더함 (1개 제거됨.) Decoy 15564건

In [ ]:
#decoy 생성을 위한 positive 뽑기

import pandas as pd
import numpy as np
import os
import random

#파일 경로 지정
n = 109 #필요한 샘플 갯수 지정
in_path = r"C:\path\to\input.csv"          # 원본 CSV
out_path = r"C:\path\to\positives_109.csv" # 결과를 다른 이름으로 저장

#input파일을 데이터프레임으로 저장
df = pd.read_csv(in_path)

# 컬럼 potency == 1인 행만 선택
label_1_data = df[df['potency'] == 1]

# 무작위로 n개 선택
random_selected_data = label_1_data.sample(n=n, random_state=42) #랜덤 시드

# 결과 저장
random_selected_data.to_csv(out_path, index=False)

In [2]:
#DUDE 파일 병합

import pandas as pd
import os

# decoy 폴더 경로
folder_path = r"C:\FTO_data\decoy\output\0812\decoy_total"

# 모든 파일 경로 가져오기
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]

combined_df = pd.DataFrame()

for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # 첫 번째 행 제거, 각 행을 tab으로 split, 2번째와 3번째 열 제거
    processed = []
    for line in lines[1:]:
        cols = line.strip().split('\t')
        # 인덱스 1과 2(2번째, 3번째 열) 제거
        filtered_cols = [col for idx, col in enumerate(cols) if idx not in [1, 2]]
        processed.append(filtered_cols)
    
    df = pd.DataFrame(processed)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# 결과 저장
combined_df.to_csv(r"C:\FTO_data\decoy\output\0812\decoy_total\combined_decoy.csv", index=False)

In [ ]:
#Canonical smiles변환

#필요 패키지 불러오기
from rdkit import Chem
import pandas as pd

#데이터 불러오기
dataset = pd.read_csv(r"C:\FTO_data\decoy\output\0812\decoy_total\combined_decoy.csv")
print("Data shape:", dataset.shape)
print(dataset.head(), "\n")

#원본 SMILES 컬럼명 바꾸기
dataset = dataset.rename(columns={'0': 'raw_SMILES'})
print("컬럼명 변경 후:", dataset.columns.tolist(), "\n")

#Canonical smiles 생성 
def canonical_smiles(smiles_list, include_stereo=True): #include_stereo: True면 입체화학(@,@@,/,\\) 정보 보존
    canonical = [] #결과를 담을 빈 리스트
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        #Mol이 None이 아니면 Moltosmiles, 아니면 None
        canonical.append(Chem.MolToSmiles(mol, isomericSmiles=include_stereo) if mol else None)
    return canonical

#canonical_SMILES컬럼을 추가하여 데이터 저장
Canon_SMILES = canonical_smiles(dataset['raw_SMILES']) #리스트 생성
assert len(Canon_SMILES) == len(dataset), "길이가 맞지 않습니다!" #원본 행 개수와 동일한지 검증 
dataset.insert(0, 'canonical_SMILES', Canon_SMILES)
print("데이터 shape (행, 열):", dataset.shape)
print(dataset[['canonical_SMILES', 'raw_SMILES']].head())

#Create a list for duplicate smiles
duplicates_smiles = dataset[dataset['canonical_SMILES'].duplicated()]['canonical_SMILES'].values
print("제거될 중복 개수:", len(duplicates_smiles))

# 중복된 행 확인 (정렬)
dup_rows = dataset[dataset['canonical_SMILES'].isin(duplicates_smiles)].sort_values(by=['canonical_SMILES'])
print("중복된 샘플들:")
print(dup_rows[['canonical_SMILES', 'raw_SMILES']].head(), "\n")

#Drop duplicate values
dataset_unique = dataset.drop_duplicates(subset=['canonical_SMILES'])
print("중복 제거 후 남은 행 수:", len(dataset_unique))
print(dataset_unique.head())

dataset_unique.to_csv(r"C:\FTO_data\decoy\output\0812\decoy_total\combined_decoy_cannon.csv", index=False)

Data shape: (15700, 1)
                                                   0
0       Cc1ccc(cc1)[N-]S(=O)(=O)c2ccc(cc2)S(=O)(=O)N
1      Cc1ccc(cc1C)[N-]S(=O)(=O)c2ccc(cc2)S(=O)(=O)N
2     c1cc(cc(c1)S(=O)(=O)N)[N-]S(=O)(=O)c2cccc(c2)F
3  c1csc(c1C(=O)NC2CC2)NC(=O)[C@H]3[C@H]4C[C@@H](...
4  c1cc2c(cc1Br)c(c[nH]2)[C@H](C(=O)[O-])[NH+]3CC... 

컬럼명 변경 후: ['raw_SMILES'] 

데이터 shape (행, 열): (15700, 2)
                                    canonical_SMILES  \
0       Cc1ccc([N-]S(=O)(=O)c2ccc(S(N)(=O)=O)cc2)cc1   
1      Cc1ccc([N-]S(=O)(=O)c2ccc(S(N)(=O)=O)cc2)cc1C   
2       NS(=O)(=O)c1cccc([N-]S(=O)(=O)c2cccc(F)c2)c1   
3  O=C(NC1CC1)c1ccsc1NC(=O)[C@@H]1[C@H](C(=O)[O-]...   
4  O=C([O-])C1CC[NH+]([C@@H](C(=O)[O-])c2c[nH]c3c...   

                                          raw_SMILES  
0       Cc1ccc(cc1)[N-]S(=O)(=O)c2ccc(cc2)S(=O)(=O)N  
1      Cc1ccc(cc1C)[N-]S(=O)(=O)c2ccc(cc2)S(=O)(=O)N  
2     c1cc(cc(c1)S(=O)(=O)N)[N-]S(=O)(=O)c2cccc(c2)F  
3  c1csc(c1C(=O)NC2CC2)NC(=O)[C@H]3[C@H]4C[C

#### 3. 최종 데이터 세트 생성

In [ ]:
#비율 조절
import pandas as pd
import os


# ----------------- 데이터 불러오기 -----------------

# 파일 경로 설정
file1 = r"C:\FTO_data\FTO_data_cannon.csv"          # 실험 데이터 (potency=1,0 등)
file2 = r"C:\FTO_data\decoy\output\0812\decoy_total\combined_decoy_cannon.csv"      # decoy 데이터셋 (potency=0)
outpath = r"C:\FTO_data"

#데이터 생성 비율 설정
ratios = [1, 3, 5, 10, 20, 50]

# CSV 파일 읽기
df1 = pd.read_csv(file1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
df2 = pd.read_csv(file2)

print("FTO_data_cannon.csv:", df1.shape)
print("FTO_DUDEdata_cannon.csv:", df2.shape)

# 열 구조 동일한지 확인
assert set(df1.columns) == set(df2.columns), "두 파일의 컬럼 구조가 다릅니다!"


FTO_data_cannon.csv: (353, 3)
FTO_DUDEdata_cannon.csv: (15564, 3)


In [3]:
# ----------------- 실측 active/inactive 분리 -----------------
act = df1[df1["potency"] == 1].copy()
ina = df1[df1["potency"] == 0].copy()

POS = len(act)       # ex) 235
NEG = len(ina)   # ex) 118

In [7]:
# ----------------- decoy 풀 준비 (모두 0 라벨로 가정) -----------------
sm_col = "canonical_SMILES" 

dec = df2.copy()
if "potency" not in dec.columns:
    dec["potency"] = 0
else:
    dec = dec[dec["potency"] == 0].copy()

# 실측과 decoy 간 SMILES 중복 제거(실측에 있는 건 decoy에서 제거)
dec_before = len(dec)
dec = dec[~dec[sm_col].isin(pd.concat([act[sm_col], ina[sm_col]]))]
print(f"[INFO] decoy duplicates removed: {dec_before - len(dec)}  (remain {len(dec)})")


[INFO] decoy duplicates removed: 0  (remain 15564)


In [8]:
# 선택적으로 'source' 컬럼 추가(분석 편의)
act["source"] = "active"
ina["source"] = "assay_inactive"
dec["source"] = "decoy"

In [9]:
# ----------------- 출력 폴더 -----------------
out_dir = os.path.join(outpath, "FTO_training")
os.makedirs(out_dir, exist_ok=True)

summary = []  # 요약 정보 저장용

# ----------------- 비율별 생성 -----------------

RANDOM_STATE = 42

# ----------------- 비율별 생성 -----------------
for r in ratios:
    target_neg = r * POS                         # 목표 음성 개수(= inactive+decoy)
    need_decoy = max(0, target_neg - NEG)    # 부족분은 decoy로 채움

    if need_decoy > len(dec):
        print(f"[WARN] ratio {r}: decoy 부족 -> {need_decoy}개 필요, {len(dec)}개만 사용")
        need_decoy = len(dec)
        
    # decoy 샘플링 (need_decoy가 0이면 빈 DF)
    dec_sample = dec.sample(n=need_decoy, random_state=RANDOM_STATE) if need_decoy > 0 else dec.iloc[0:0]

    # 합치기: active 전부 + inactive 전부 + decoy 일부
    df_ratio = pd.concat([act, ina, dec_sample], ignore_index=True)

    # 저장
    out_file = os.path.join(out_dir, f"FTO_training_{r}x.csv")
    df_ratio.to_csv(out_file, index=False, encoding="utf-8-sig")

    # 로그 & 요약 수집
    print(f"[SAVE] {out_file} -> pos={POS}, neg(real)={NEG}, decoy={len(dec_sample)}, total={len(df_ratio)}")
    summary.append({
        "ratio_neg_per_pos": f"1:{r}",
        "pos": POS,
        "neg_real": NEG,
        "decoy_used": len(dec_sample),
        "total": len(df_ratio),
        "path": out_file
    })

# ----------------- 요약표 출력 -----------------
summary_df = pd.DataFrame(summary)
print("\n=== Dataset summary by ratio ===")
print(summary_df)

[SAVE] C:\FTO_data\FTO_training\FTO_training_1x.csv -> pos=235, neg(real)=118, decoy=117, total=470
[SAVE] C:\FTO_data\FTO_training\FTO_training_3x.csv -> pos=235, neg(real)=118, decoy=587, total=940
[SAVE] C:\FTO_data\FTO_training\FTO_training_5x.csv -> pos=235, neg(real)=118, decoy=1057, total=1410
[SAVE] C:\FTO_data\FTO_training\FTO_training_10x.csv -> pos=235, neg(real)=118, decoy=2232, total=2585
[SAVE] C:\FTO_data\FTO_training\FTO_training_20x.csv -> pos=235, neg(real)=118, decoy=4582, total=4935
[SAVE] C:\FTO_data\FTO_training\FTO_training_50x.csv -> pos=235, neg(real)=118, decoy=11632, total=11985

=== Dataset summary by ratio ===
  ratio_neg_per_pos  pos  neg_real  decoy_used  total  \
0               1:1  235       118         117    470   
1               1:3  235       118         587    940   
2               1:5  235       118        1057   1410   
3              1:10  235       118        2232   2585   
4              1:20  235       118        4582   4935   
5          

#### 4. Fingerprint생성

In [1]:
## 라이브러리 임포트 ##
import os                         # 파일 경로 및 디렉토리 작업을 위한 표준 라이브러리
import re                         # 정규표현식 처리 (문자열 패턴 검색 등)
import glob                       # 파일 경로에서 와일드카드 사용 가능하게 하는 모듈 (ex. *.csv 찾기)
import pickle                     # 파이썬 객체 저장 및 불러오기 (모델 저장/로드)
import pandas as pd               # 데이터프레임 생성 및 CSV/Excel 입출력 등 데이터 처리
import numpy as np                # 수치 계산 및 배열 처리 (벡터/행렬 등)
import multiprocessing            # 병렬 처리 (코어 여러 개 사용)
from collections import Counter   # 리스트 등에서 항목 개수 세기 (빈도 계산)

# 병렬처리 관련 라이브러리들
from concurrent.futures import ProcessPoolExecutor, as_completed  # 고수준 병렬처리 API
from joblib import Parallel, delayed                               # sklearn과 사용하기 좋은 병렬처리 도구
from tqdm import tqdm               # 반복문 진행 상황 시각화 (진행바 출력)

# RDKit: 화합물 처리 및 fingerprint 계산
from rdkit import Chem, DataStructs                      # SMILES → Mol, Mol 간 유사도 계산 등
from rdkit.Chem import AllChem, MACCSkeys, RDKFingerprint  # ECFP4, MACCS, RDKit FP 계산

# Mordred: 분자 descriptor 계산 라이브러리
from mordred import Calculator, descriptors              #Molecular descriptor 계산

In [5]:
##Fingerprint 생성 함수 (ECFP4)##

# 개별 SMILES → ECFP4 fingerprint (bit-vector 리스트)
def smiles_to_fp(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
    arr = np.zeros((nBits,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# 병렬처리 포함 전처리 함수
def process_smiles_fingerprints(smiles_list, radius=2, nBits=1024, workers=None, show_progress=True):
    # 병렬 작업에 사용할 워커 수 설정 (사용 가능한 CPU 코어 수-4)
    if workers is None:
        total_cores = multiprocessing.cpu_count()
        workers = max(1, total_cores - 4)

    # 결측값(NaN) 제거: SMILES 값이 있는 것만 필터링
    smiles_list_cleaned = [s for s in smiles_list if pd.notnull(s)]

    # tqdm을 사용해 진행률 표시 여부 설정
    iterator = tqdm(smiles_list_cleaned) if show_progress else smiles_list_cleaned

    # 병렬 처리로 SMILES → fingerprint 변환 수행
    fingerprints = Parallel(n_jobs=workers)(delayed(smiles_to_fp)(smi) for smi in iterator)

    # 변환 결과 중 None이 아닌 유효한 SMILES만 추출
    valid_data = [(smi, fp) for smi, fp in zip(smiles_list_cleaned, fingerprints) if fp is not None]

    # 유효한 결과가 하나도 없을 경우 빈 DataFrame 반환
    if not valid_data:
        print("유효한 fingerprint가 없습니다.")
        return pd.DataFrame()

    # 유효한 SMILES와 해당 fingerprint 배열 분리
    valid_smiles, valid_fps = zip(*valid_data)

    # fingerprint의 컬럼명을  1x ~ nBits x 형식으로 지정
    bit_columns = [f"X{i+1}" for i in range(nBits)]
    
    # 결과를 DataFrame으로 구성
    fp_df = pd.DataFrame(valid_fps, columns=bit_columns)
    fp_df.insert(0, 'SMILES', valid_smiles)  # 첫 번째 열에 SMILES 추가

    # 결과 요약 출력
    print(f"총 {len(smiles_list)}개의 SMILES 중 {len(valid_smiles)}개가 정상적으로 fingerprint로 변환되었습니다.")

    return fp_df

In [ ]:
## 호출부

folder_path = r"C:\FTO_data\decoy\output\0812\decoy_total"

# 반복 처리
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)

    # CSV 불러오기
    df = pd.read_csv(file_path)

    # 유효한 SMILES만 추출
    smiles_col = 'canonical_SMILES'
    df_clean = df[pd.notnull(df[smiles_col])].reset_index(drop=True)
    smiles_list = df_clean[smiles_col].tolist()

    # Fingerprint 생성
    fingerprint_df = process_smiles_fingerprints(smiles_list)

    # SMILES 컬럼 제거 후 원본 메타데이터와 fingerprint 결합
    fingerprint_only = fingerprint_df.drop(columns=['SMILES'])  # 중복 방지

    # 최종 결합 (원본 df_clean + fingerprint)
    final_df = pd.concat([df_clean, fingerprint_only], axis=1)

    # 결과 shape 출력
    print(f"📁 처리된 파일: {file_name}")
    print("📊 최종 결과 shape:", final_df.shape)

    # 결과 저장
    output_file_name = os.path.splitext(file_name)[0] + "_ecfp4.csv"
    output_path = os.path.join(folder_path, output_file_name)
    final_df.to_csv(output_path, index=False)
    print(f"✅ 저장 완료: {output_path}\n")

100%|██████████| 2585/2585 [00:00<00:00, 8069.46it/s]


총 2585개의 SMILES 중 2585개가 정상적으로 fingerprint로 변환되었습니다.
📁 처리된 파일: FTO_training_10x.csv
📊 최종 결과 shape: (2585, 1028)
✅ 저장 완료: C:\FTO_data\FTO_training\FTO_training_10x_ecfp4.csv



100%|██████████| 470/470 [00:00<00:00, 2458.33it/s]


총 470개의 SMILES 중 470개가 정상적으로 fingerprint로 변환되었습니다.
📁 처리된 파일: FTO_training_1x.csv
📊 최종 결과 shape: (470, 1028)
✅ 저장 완료: C:\FTO_data\FTO_training\FTO_training_1x_ecfp4.csv



100%|██████████| 4935/4935 [00:00<00:00, 7989.33it/s] 


...

총 940개의 SMILES 중 940개가 정상적으로 fingerprint로 변환되었습니다.
📁 처리된 파일: FTO_training_3x.csv
📊 최종 결과 shape: (940, 1028)
✅ 저장 완료: C:\FTO_data\FTO_training\FTO_training_3x_ecfp4.csv



100%|██████████| 11985/11985 [00:00<00:00, 15162.78it/s]


총 11985개의 SMILES 중 11985개가 정상적으로 fingerprint로 변환되었습니다.
📁 처리된 파일: FTO_training_50x.csv
📊 최종 결과 shape: (11985, 1028)
✅ 저장 완료: C:\FTO_data\FTO_training\FTO_training_50x_ecfp4.csv



100%|██████████| 1410/1410 [00:00<00:00, 4589.69it/s]


총 1410개의 SMILES 중 1410개가 정상적으로 fingerprint로 변환되었습니다.
📁 처리된 파일: FTO_training_5x.csv
📊 최종 결과 shape: (1410, 1028)
✅ 저장 완료: C:\FTO_data\FTO_training\FTO_training_5x_ecfp4.csv



### [모델학습]

#### 1. 데이터 불러오기 (ECFP4 fingerprint)

In [2]:
## 라이브러리 임포트 ##
from sklearn.model_selection import train_test_split 
import logging
import os
from datetime import datetime
import pandas as pd

In [5]:
##환경설정##

#로깅 설정
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s] %(name)s: %(message)s')

#경로지정
INPATH=r"C:\FTO_data\FTO_training\data_20250813\fingerprint"
today = datetime.today().strftime('%Y%m%d') 
OUTPATH = os.path.join(r"C:\FTO_data\FTO_training", f"data_{today}")
if not os.path.exists(OUTPATH):
    os.makedirs(OUTPATH)

In [6]:
# 처리할 파일 목록
file_list = [f for f in os.listdir(INPATH) if f.lower().endswith('.csv')]

# 정규식: 접미사만 추출할 때 사용할 패턴
suffix_pattern = re.compile(r'_(\d+x)')

# 최종 데이터들을 저장할 딕셔너리
train_dfs = {} # 모델링에 사용할 데이터 (입력 + 타겟)
metadata_dfs = {} # 메타데이터

for file_name in file_list:
    file_path = os.path.join(INPATH, file_name)
    logging.info(f"🔍 처리 중: {file_name}")

    try:
        # 1. 파일 불러오기
        df = pd.read_csv(file_path)
        
        # 2. 메타데이터로 사용할 열 이름 정의
        meta_cols = ['canonical_SMILES', 'raw_SMILES', 'source']
        
        # 3. 데이터 분리
        # 메타데이터 추출
        df_meta = df[meta_cols].copy()
        
        # 모델링용 데이터 추출: 전체 열에서 메타데이터 열만 제외
        train_cols = df.columns.drop(meta_cols)
        df_train = df[train_cols].copy()
    
        # 4. 결과 저장
        # 파일 이름에서 접미사(1x 등) 추출
        match = suffix_pattern.search(file_name)
        suffix = match.group(1) if match else 'unknown'

        # 각 딕셔너리에 접미사를 키로 하여 데이터프레임 저장
        train_dfs[f'df_train_{suffix}'] = df_train
        metadata_dfs[f'metadata_{suffix}'] = df_meta

        logging.info(f"✅ 데이터 분리 완료: {file_name}")
        logging.info(f"  -> df_train_{suffix}: {df_train.shape}")
        logging.info(f"  -> metadata_{suffix}: {df_meta.shape}")

        # globals()를 사용해 직접 변수 생성
        globals()[f'df_train_{suffix}'] = df_train
        globals()[f'metadata_{suffix}'] = df_meta

    except KeyError as e:
        logging.error(f"❌ 처리 실패: {file_name} - 열 이름 확인 필요: {e}")
    except Exception as e:
        logging.error(f"❌ 처리 실패: {file_name} - {str(e)}")

2025-08-27 10:04:19,910 [INFO] root: 🔍 처리 중: FTO_training_10x_ecfp4.csv
2025-08-27 10:04:20,187 [INFO] root: ✅ 데이터 분리 완료: FTO_training_10x_ecfp4.csv
2025-08-27 10:04:20,195 [INFO] root:   -> df_train_10x: (2585, 1025)
2025-08-27 10:04:20,195 [INFO] root:   -> metadata_10x: (2585, 3)
2025-08-27 10:04:20,195 [INFO] root: 🔍 처리 중: FTO_training_1x_ecfp4.csv
2025-08-27 10:04:20,264 [INFO] root: ✅ 데이터 분리 완료: FTO_training_1x_ecfp4.csv
2025-08-27 10:04:20,267 [INFO] root:   -> df_train_1x: (470, 1025)
2025-08-27 10:04:20,267 [INFO] root:   -> metadata_1x: (470, 3)
2025-08-27 10:04:20,267 [INFO] root: 🔍 처리 중: FTO_training_20x_ecfp4.csv
2025-08-27 10:04:20,750 [INFO] root: ✅ 데이터 분리 완료: FTO_training_20x_ecfp4.csv
2025-08-27 10:04:20,750 [INFO] root:   -> df_train_20x: (4935, 1025)
2025-08-27 10:04:20,750 [INFO] root:   -> metadata_20x: (4935, 3)
2025-08-27 10:04:20,750 [INFO] root: 🔍 처리 중: FTO_training_3x_ecfp4.csv
2025-08-27 10:04:20,890 [INFO] root: ✅ 데이터 분리 완료: FTO_training_3x_ecfp4.csv
2025-08

In [7]:
df_train_1x

,potency,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X1015,X1016,X1017,X1018,X1019,X1020,X1021,X1022,X1023,X1024
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
466,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
467,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split
import logging

# 분리 대상 접미사
data_suffixes = ['1x', '3x', '5x', '10x', '20x', '50x']

for suffix in data_suffixes:
    try:
        # 1. 이전 단계에서 만든 df_train_{suffix} 변수를 불러옵니다.
        df_full = globals()[f'df_train_{suffix}']

        # 2. 불러온 데이터프레임에서 X와 Y를 분리합니다.
        y_col = 'potency'
        x_cols = df_full.columns.drop(y_col)
        
        x_full = df_full[x_cols]
        y_full = df_full[y_col]

        # 3. train/test 분리를 수행합니다.
        x_train, x_test, y_train, y_test = train_test_split(
            x_full, y_full,
            test_size=0.1,
            random_state=42,
            stratify=y_full
        )

        # 4. 분리된 결과를 새로운 변수로 저장합니다.
        globals()[f'x_train_{suffix}'] = x_train
        globals()[f'y_train_{suffix}'] = y_train
        globals()[f'x_test_{suffix}'] = x_test
        globals()[f'y_test_{suffix}'] = y_test
        
        logging.info(
            f"✅ {suffix} 데이터 → train/test 분리 완료: "
            f"x_train{x_train.shape}, y_train{y_train.shape}, "
            f"x_test{x_test.shape}, y_test{y_test.shape}"
        )
        
    except KeyError:
        logging.error(f"❌ 변수 로드 실패: df_train_{suffix}를 찾을 수 없습니다.")
    except Exception as e:
        logging.error(f"❌ {suffix} 데이터 분리 실패: {str(e)}")

2025-08-27 10:05:04,439 [INFO] root: ✅ 1x 데이터 → train/test 분리 완료: x_train(423, 1024), y_train(423,), x_test(47, 1024), y_test(47,)
2025-08-27 10:05:04,454 [INFO] root: ✅ 3x 데이터 → train/test 분리 완료: x_train(846, 1024), y_train(846,), x_test(94, 1024), y_test(94,)
2025-08-27 10:05:04,471 [INFO] root: ✅ 5x 데이터 → train/test 분리 완료: x_train(1269, 1024), y_train(1269,), x_test(141, 1024), y_test(141,)
2025-08-27 10:05:04,502 [INFO] root: ✅ 10x 데이터 → train/test 분리 완료: x_train(2326, 1024), y_train(2326,), x_test(259, 1024), y_test(259,)
2025-08-27 10:05:04,552 [INFO] root: ✅ 20x 데이터 → train/test 분리 완료: x_train(4441, 1024), y_train(4441,), x_test(494, 1024), y_test(494,)
2025-08-27 10:05:04,673 [INFO] root: ✅ 50x 데이터 → train/test 분리 완료: x_train(10786, 1024), y_train(10786,), x_test(1199, 1024), y_test(1199,)


#### 2. 모델학습 (PyCaret)

In [10]:
from pycaret.classification import *

In [64]:
#데이터 전처리 및 모델학습

# 모델 비교 결과를 데이터프레임으로 저장할 딕셔너리
all_comparison_df = {} 
data_suffixes = ['1x', '3x', '5x', '10x', '20x', '50x']

for suffix in data_suffixes:
    x_train = globals()[f'x_train_{suffix}']
    y_train = globals()[f'y_train_{suffix}']
    df_train = pd.concat([x_train, y_train], axis=1)

    # 데이터 셋업
    setup(
        data=df_train,
        target='potency',
        session_id=42,
        train_size=0.9,
        fold=10, #디폴트
        normalize=True,
        fix_imbalance=True,
        remove_outliers=True,
        n_jobs=-1, 
        verbose=False
    )

    print(f"✅ [{suffix}] 모델 비교 시작")    
    
    # 모델 비교
    compare_models(fold=5, sort='F1', n_select=15, turbo=True)
    
    # [핵심] 결과를 pull()을 이용해 데이터프레임으로 저장합니다.
    results_df = pull()
    all_comparison_df[suffix] = results_df
    
    print(f"✅ [{suffix}] 결과 저장 완료")

print("\n모든 데이터세트의 모델 비교 결과(데이터프레임) 저장이 완료되었습니다!")

2025-08-27 11:52:35,398 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


✅ [1x] 모델 비교 시작


2025-08-27 11:52:35,401 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8474,0.9327,0.8053,0.8873,0.8395,0.6947,0.7040,0.2900
lr,Logistic Regression,0.8237,0.9125,0.8421,0.8227,0.8287,0.6474,0.6536,1.7620
lightgbm,Light Gradient Boosting Machine,0.8237,0.9271,0.8368,0.8192,0.8265,0.6474,0.6498,0.2700
rf,Random Forest Classifier,0.8316,0.9319,0.7947,0.8725,0.8255,0.6632,0.6743,0.3440
ridge,Ridge Classifier,0.8079,0.8748,0.8421,0.7904,0.8139,0.6158,0.6200,0.2940
svm,SVM - Linear Kernel,0.8000,0.8843,0.8053,0.7991,0.7992,0.6000,0.6054,0.3400
gbc,Gradient Boosting Classifier,0.8000,0.8944,0.7895,0.8149,0.7983,0.6000,0.6058,0.3880
ada,Ada Boost Classifier,0.7921,0.8564,0.8000,0.7921,0.7915,0.5842,0.5908,0.4320
knn,K Neighbors Classifier,0.8079,0.8859,0.7211,0.8913,0.7871,0.6158,0.6374,1.1100
dt,Decision Tree Classifier,0.7763,0.7763,0.7789,0.7790,0.7757,0.5526,0.5572,0.3060


2025-08-27 11:52:35,407 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


...

2025-08-27 12:00:51,839 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9936,0.9850,0.7684,0.8918,0.8242,0.8210,0.8240,3.1320
lr,Logistic Regression,0.9923,0.9782,0.8474,0.7813,0.8119,0.8080,0.8093,1.6420
et,Extra Trees Classifier,0.9930,0.9851,0.7579,0.8670,0.8081,0.8045,0.8068,2.2460
gbc,Gradient Boosting Classifier,0.9919,0.9776,0.8263,0.7757,0.7987,0.7946,0.7957,7.6840
rf,Random Forest Classifier,0.9923,0.9850,0.6842,0.8963,0.7752,0.7714,0.7791,2.8480
ridge,Ridge Classifier,0.9897,0.9789,0.8947,0.6835,0.7738,0.7687,0.7766,2.4500
lda,Linear Discriminant Analysis,0.9897,0.9789,0.8947,0.6835,0.7738,0.7687,0.7766,3.7580
ada,Ada Boost Classifier,0.9873,0.9623,0.7474,0.6567,0.6973,0.6909,0.6933,4.7120
dt,Decision Tree Classifier,0.9832,0.8264,0.6632,0.5609,0.6074,0.5989,0.6012,1.9140
svm,SVM - Linear Kernel,0.9884,0.9788,0.4368,0.9272,0.5905,0.5855,0.6301,2.3480


2025-08-27 12:00:51,841 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


2025-08-27 12:04:32,978 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 12:04:32,980 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 12:04:38,936 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 12:04:38,938 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 12:04:42,613 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 12:04:42,615 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 12:04:47,434 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 12:04:47,437 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


✅ [50x] 결과 저장 완료

모든 데이터세트의 모델 비교 결과(데이터프레임) 저장이 완료되었습니다!


In [65]:
##상위 모델 추출
top_models_set = set()

# 저장된 결과 데이터프레임에서 직접 모델 ID를 추출합니다.
for suffix, results_df in all_comparison_df.items():
    
    # [핵심] 데이터프레임의 인덱스가 바로 모델 ID입니다. 상위 5개를 가져옵니다.
    top_5_ids = results_df.index[:5].tolist()
    
    # .update()를 사용해 리스트의 모든 항목을 set에 추가
    top_models_set.update(top_5_ids)

final_tuning_list = list(top_models_set)

print("✅ 튜닝 대상 모델 목록이 생성되었습니다.")
print(f" -> 최종 튜닝 모델: {final_tuning_list}")

✅ 튜닝 대상 모델 목록이 생성되었습니다.
 -> 최종 튜닝 모델: ['lr', 'rf', 'gbc', 'lightgbm', 'ridge', 'et']


In [ ]:
##하이퍼파라미터 튜닝

# 결과를 저장할 최종 딕셔너리 (사용자 요청 형식)
all_model_dfs = {}    # 모델별 전체 fold 성능 저장
all_summary_dfs = {}  # 데이터세트별로 각 모델의 성능 요약
all_tuned_models = {}   # 튜닝된 모델 객체 저장

data_suffixes = ['1x', '3x', '5x', '10x', '20x', '50x']

print("=============== 하이퍼파라미터 튜닝 및 최종 결과 저장 시작 ===============")

for suffix in data_suffixes:
    print(f"\033[1;30m\n------------ 데이터세트 [{suffix}] 튜닝 시작 ------------\033[0m")
    
    # 데이터 준비
    x_train = globals()[f'x_train_{suffix}']
    y_train = globals()[f'y_train_{suffix}']
    df_train = pd.concat([x_train, y_train], axis=1)
    
    # PyCaret 설정
    data_setup = setup(
        data=df_train,
        target='potency',
        session_id=42,
        train_size=0.9,
        fold=10, #디폴트
        normalize=True,
        fix_imbalance=True,
        remove_outliers=True,
        n_jobs=-1,
        verbose=False
    )
    
    # 현재 suffix의 결과를 임시 저장할 변수들
    model_dfs = {}
    tuned_models = {} 
    summary_rows = []
    
    # 모델 튜닝
    for model_id in final_tuning_list:
        try:
            print(f"\033[1;31m\n  -> 모델 [{model_id}] 튜닝 중...\033[0m")
            
            # 튜닝할 모델 생성
            model_to_tune = create_model(model_id, verbose=False)
            
            # 모델 튜닝
            tuned_model = tune_model(
                model_to_tune,
                optimize='F1', 
                n_iter=100,
                verbose=True)
            
            result_grid = pull().copy()
            
            # 모델 이름 추출 (예: 'ExtraTreesClassifier')
            model_name = str(tuned_model).split('(')[0]
            
            # Mean, Std 행을 나중에 식별하기 위해 'Type' 열 추가
            result_grid['Model'] = model_name
            result_grid['Type'] = result_grid.index
            
            # 각 딕셔너리에 결과 저장
            model_dfs[model_name] = result_grid
            tuned_models[model_name] = tuned_model
            summary_rows.append(result_grid[result_grid['Type'].isin(['Mean', 'Std'])])
            
            print(f"  ✅ 튜닝 완료!")
            
        except Exception as e:
            logging.error(f"  ❌ 모델 [{model_id}] 튜닝 실패: {e}")
            
    # 현재 suffix의 요약(summary) 데이터프레임 생성
    summary_df = pd.concat(summary_rows, ignore_index=True)
    
    # 최종 딕셔너리에 현재 suffix의 결과들을 저장
    all_model_dfs[suffix] = model_dfs
    all_summary_dfs[suffix] = summary_df
    all_tuned_models[suffix] = tuned_models

    print(f"\n[완료] {suffix} 튜닝 및 저장 ✅")
    
#모델 및 평가지표 저장
print("\n🎉 모든 튜닝 및 저장 작업이 완료되었습니다!")

=============== 하이퍼파라미터 튜닝 및 최종 결과 저장 시작 ===============

------------ 데이터세트 [1x] 튜닝 시작 ------------


2025-08-27 12:06:24,210 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 12:06:24,210 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.



  -> 모델 [lr] 튜닝 중...


2025-08-27 12:06:27,084 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


2025-08-27 12:06:27,105 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9949,0.9823,0.8947,0.8500,0.8718,0.8692,0.8695
1,0.9918,0.9715,0.8421,0.7619,0.8000,0.7958,0.7968
2,0.9907,0.9916,0.9474,0.6923,0.8000,0.7954,0.8056
3,0.9918,0.9771,0.7368,0.8235,0.7778,0.7736,0.7748
4,0.9938,0.9823,0.8947,0.8095,0.8500,0.8469,0.8480
5,0.9897,0.9879,0.7368,0.7368,0.7368,0.7316,0.7316
6,0.9856,0.9894,0.5789,0.6471,0.6111,0.6038,0.6048
7,0.9969,0.9969,0.9474,0.9000,0.9231,0.9215,0.9218
8,0.9887,0.9826,0.7895,0.6818,0.7317,0.7259,0.7280


2025-08-27 19:58:05,615 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


2025-08-27 20:54:52,927 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.
2025-08-27 20:54:52,932 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
  ✅ 튜닝 완료!

[완료] 50x 튜닝 및 저장 ✅

🎉 모든 튜닝 및 저장 작업이 완료되었습니다!


In [70]:
## 튜닝 결과비교

metric = "F1"  # 정렬 기준 (원하면 "AUC", "Accuracy" 등으로 변경)
cols = ["Model", "Accuracy", "AUC", "Recall", "Prec.", "F1", "Kappa", "MCC"]  # 보기용 컬럼

for suffix in data_suffixes:
    print(f"\033[1;30m\n[{suffix}] 결과\033[0m")

    # 1) 튜닝 전: 비교 결과 상위 5개
    print("하이퍼파라미터 튜닝 전 (Top 5)")
    results_df = all_comparison_df[suffix]
    # 정렬 기준 컬럼이 없을 수도 있으므로 방어
    if metric in results_df.columns:
        pre_top5 = results_df.sort_values(by=metric, ascending=False).head(5)
    else:
        pre_top5 = results_df

    display(pre_top5[cols] if set(cols).issubset(pre_top5.columns) else pre_top5)

    # 2) 튜닝 후: Mean 행만 필터 → 상위 5개
    print("하이퍼파라미터 튜닝 후")
    summary_df = all_summary_dfs[suffix]
    post_df = summary_df[summary_df["Type"] == "Mean"].copy()

    if post_df.empty:
        print(" - [알림] 'Mean' 행이 없습니다. summary_df 전체를 표시합니다.")
        display(summary_df)
        continue

    if metric in post_df.columns:
        post_top5 = post_df.sort_values(by=metric, ascending=False) #.head(6)
    else:
        post_top5 = post_df

    display(post_top5[cols] if set(cols).issubset(post_top5.columns) else post_top5)


[1x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
et,Extra Trees Classifier,0.8474,0.9327,0.8053,0.8873,0.8395,0.6947,0.7040
lr,Logistic Regression,0.8237,0.9125,0.8421,0.8227,0.8287,0.6474,0.6536
lightgbm,Light Gradient Boosting Machine,0.8237,0.9271,0.8368,0.8192,0.8265,0.6474,0.6498
rf,Random Forest Classifier,0.8316,0.9319,0.7947,0.8725,0.8255,0.6632,0.6743
ridge,Ridge Classifier,0.8079,0.8748,0.8421,0.7904,0.8139,0.6158,0.6200


하이퍼파라미터 튜닝 후


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
4,GradientBoostingClassifier,0.8579,0.9253,0.8579,0.8595,0.8574,0.7158,0.7181
6,LGBMClassifier,0.8526,0.9097,0.8421,0.8672,0.8512,0.7053,0.7102
2,RandomForestClassifier,0.8500,0.9324,0.8000,0.9014,0.8406,0.7000,0.7130
0,LogisticRegression,0.8368,0.9183,0.8421,0.8408,0.8390,0.6737,0.6777
10,ExtraTreesClassifier,0.8342,0.8920,0.8421,0.8328,0.8353,0.6684,0.6722
8,RidgeClassifier,0.8184,0.8737,0.8632,0.7974,0.8267,0.6368,0.6430



[3x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
4,GradientBoostingClassifier,0.9847,0.9741,0.7895,0.8829,0.8292,0.8213,0.8250
6,LGBMClassifier,0.9847,0.9741,0.7632,0.9045,0.8252,0.8173,0.8219
0,LogisticRegression,0.9830,0.9694,0.8263,0.8207,0.8191,0.8102,0.8126
10,ExtraTreesClassifier,0.9830,0.9776,0.8000,0.8391,0.8160,0.8071,0.8090
2,RandomForestClassifier,0.9825,0.9726,0.7684,0.8532,0.8052,0.7961,0.7991
8,RidgeClassifier,0.9752,0.9662,0.8579,0.7014,0.7680,0.7552,0.7615



[50x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
lightgbm,Light Gradient Boosting Machine,0.9936,0.9850,0.7684,0.8918,0.8242,0.8210,0.8240
lr,Logistic Regression,0.9923,0.9782,0.8474,0.7813,0.8119,0.8080,0.8093
et,Extra Trees Classifier,0.9930,0.9851,0.7579,0.8670,0.8081,0.8045,0.8068
gbc,Gradient Boosting Classifier,0.9919,0.9776,0.8263,0.7757,0.7987,0.7946,0.7957
rf,Random Forest Classifier,0.9923,0.9850,0.6842,0.8963,0.7752,0.7714,0.7791


하이퍼파라미터 튜닝 후


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
6,LGBMClassifier,0.9940,0.9783,0.7947,0.8973,0.8348,0.8318,0.8375
0,LogisticRegression,0.9929,0.9800,0.8579,0.8046,0.8270,0.8234,0.8256
4,GradientBoostingClassifier,0.9933,0.9825,0.8053,0.8457,0.8219,0.8185,0.8203
10,ExtraTreesClassifier,0.9919,0.9841,0.8211,0.7792,0.7967,0.7926,0.7943
2,RandomForestClassifier,0.9923,0.9844,0.7737,0.8233,0.7944,0.7904,0.7926
8,RidgeClassifier,0.9893,0.9771,0.8895,0.6786,0.7667,0.7613,0.7703


In [77]:
## 튜닝 결과비교

metric = "F1"  # 정렬 기준

for suffix in data_suffixes:
    print(f"\033[1;30m\n[{suffix}] 결과\033[0m")

    # --- 1) 튜닝 전: 비교 결과 상위 5개 ---
    print("하이퍼파라미터 튜닝 전 (Top 5)")
    results_df = all_comparison_df[suffix]
    pre_top5 = results_df.sort_values(by=metric, ascending=False).head(5)
    
    # [수정] 'Model' 열을 인덱스로 만들어서 출력합니다.
    display(pre_top5.set_index('Model'))

    # --- 2) 튜닝 후: Mean 행만 필터 → 상위 5개 ---
    print("\n하이퍼파라미터 튜닝 후 (Top 5)")
    summary_df = all_summary_dfs[suffix]
    post_df = summary_df[summary_df["Type"] == "Mean"].copy()

    if post_df.empty:
        print(" - [알림] 'Mean' 행이 없습니다.")
        continue

    # [수정] 'Model' 열을 인덱스로 설정합니다.
    post_df.set_index('Model', inplace=True)
    
    post_top5 = post_df.sort_values(by=metric, ascending=False).head(5)
    display(post_top5)


[1x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
Model,,,,,,,,
Extra Trees Classifier,0.8474,0.9327,0.8053,0.8873,0.8395,0.6947,0.7040,0.290
Logistic Regression,0.8237,0.9125,0.8421,0.8227,0.8287,0.6474,0.6536,1.762
Light Gradient Boosting Machine,0.8237,0.9271,0.8368,0.8192,0.8265,0.6474,0.6498,0.270
Random Forest Classifier,0.8316,0.9319,0.7947,0.8725,0.8255,0.6632,0.6743,0.344
Ridge Classifier,0.8079,0.8748,0.8421,0.7904,0.8139,0.6158,0.6200,0.294



하이퍼파라미터 튜닝 후 (Top 5)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Type
Model,,,,,,,,
GradientBoostingClassifier,0.8579,0.9253,0.8579,0.8595,0.8574,0.7158,0.7181,Mean
LGBMClassifier,0.8526,0.9097,0.8421,0.8672,0.8512,0.7053,0.7102,Mean
RandomForestClassifier,0.8500,0.9324,0.8000,0.9014,0.8406,0.7000,0.7130,Mean
LogisticRegression,0.8368,0.9183,0.8421,0.8408,0.8390,0.6737,0.6777,Mean
ExtraTreesClassifier,0.8342,0.8920,0.8421,0.8328,0.8353,0.6684,0.6722,Mean



[3x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Type
Model,,,,,,,,
GradientBoostingClassifier,0.9847,0.9741,0.7895,0.8829,0.8292,0.8213,0.8250,Mean
LGBMClassifier,0.9847,0.9741,0.7632,0.9045,0.8252,0.8173,0.8219,Mean
LogisticRegression,0.9830,0.9694,0.8263,0.8207,0.8191,0.8102,0.8126,Mean
ExtraTreesClassifier,0.9830,0.9776,0.8000,0.8391,0.8160,0.8071,0.8090,Mean
RandomForestClassifier,0.9825,0.9726,0.7684,0.8532,0.8052,0.7961,0.7991,Mean



[50x] 결과
하이퍼파라미터 튜닝 전 (Top 5)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
Model,,,,,,,,
Light Gradient Boosting Machine,0.9936,0.9850,0.7684,0.8918,0.8242,0.8210,0.8240,3.132
Logistic Regression,0.9923,0.9782,0.8474,0.7813,0.8119,0.8080,0.8093,1.642
Extra Trees Classifier,0.9930,0.9851,0.7579,0.8670,0.8081,0.8045,0.8068,2.246
Gradient Boosting Classifier,0.9919,0.9776,0.8263,0.7757,0.7987,0.7946,0.7957,7.684
Random Forest Classifier,0.9923,0.9850,0.6842,0.8963,0.7752,0.7714,0.7791,2.848



하이퍼파라미터 튜닝 후 (Top 5)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Type
Model,,,,,,,,
LGBMClassifier,0.9940,0.9783,0.7947,0.8973,0.8348,0.8318,0.8375,Mean
LogisticRegression,0.9929,0.9800,0.8579,0.8046,0.8270,0.8234,0.8256,Mean
GradientBoostingClassifier,0.9933,0.9825,0.8053,0.8457,0.8219,0.8185,0.8203,Mean
ExtraTreesClassifier,0.9919,0.9841,0.8211,0.7792,0.7967,0.7926,0.7943,Mean
RandomForestClassifier,0.9923,0.9844,0.7737,0.8233,0.7944,0.7904,0.7926,Mean


In [74]:
import joblib

In [ ]:
#모델 및 평가지표 저장
# 결과를 저장할 기본 경로 설정 (이전 코드와 동일하게)
today = datetime.today().strftime('%Y%m%d')
OUTPATH = os.path.join(r"C:\FTO_data\FTO_training", f"data_{today}")
os.makedirs(OUTPATH, exist_ok=True)

print(f"결과를 다음 경로에 저장합니다: {OUTPATH}")

# Cell 3에서 생성된 최종 결과 딕셔너리들을 순회합니다.
for suffix in all_model_dfs.keys():
    
    # 1. 각 데이터세트별로 결과 폴더 생성
    suffix_path = os.path.join(OUTPATH, f"{suffix}_fp") ##폴더명: fingerprint 데이터 사용함.
    os.makedirs(suffix_path, exist_ok=True)
    print(f"\n--- [{suffix}] 데이터세트 결과 저장 중 ---")

    # 2. 요약 결과(summary_df) 저장
    summary_df = all_summary_dfs[suffix]
    summary_path = os.path.join(suffix_path, f"{suffix}_summary_evaluation.csv")
    summary_df.to_csv(summary_path, index=False)
    print(f"  -> 요약 결과 저장 완료: {summary_path}")

    # 3. 모델별 상세 평가결과(evaluation.csv) 저장
    model_dfs = all_model_dfs[suffix]
    for model_name, df in model_dfs.items():
        eval_path = os.path.join(suffix_path, f"{suffix}_{model_name}_evaluation.csv")
        df.to_csv(eval_path, index=False)
    print(f"  -> 모델별 상세 결과 {len(model_dfs)}개 저장 완료.")

    # 4. 튜닝된 모델(.pkl) 저장
    tuned_models = all_tuned_models[suffix]
    model_save_path = os.path.join(suffix_path, "models")
    os.makedirs(model_save_path, exist_ok=True)
    
    for model_name, model_object in tuned_models.items():
        model_path = os.path.join(model_save_path, f"{suffix}_{model_name}_model.pkl")
        joblib.dump(model_object, model_path)
    print(f"  -> 튜닝된 모델 {len(tuned_models)}개 저장 완료.")

print("\n🎉 모든 결과 파일 저장이 완료되었습니다!")

결과를 다음 경로에 저장합니다: C:\FTO_data\FTO_training\data_20250827

--- [1x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250827\1x_fp\1x_summary_evaluation.csv
  -> 모델별 상세 결과 6개 저장 완료.
  -> 튜닝된 모델 6개 저장 완료.

--- [3x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250827\3x_fp\3x_summary_evaluation.csv
  -> 모델별 상세 결과 6개 저장 완료.
  -> 튜닝된 모델 6개 저장 완료.

--- [5x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250827\5x_fp\5x_summary_evaluation.csv
  -> 모델별 상세 결과 6개 저장 완료.
  -> 튜닝된 모델 6개 저장 완료.

--- [10x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250827\10x_fp\10x_summary_evaluation.csv
  -> 모델별 상세 결과 6개 저장 완료.
  -> 튜닝된 모델 6개 저장 완료.

--- [20x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250827\20x_fp\20x_summary_evaluation.csv
  -> 모델별 상세 결과 6개 저장 완료.
  -> 튜닝된 모델 6개 저장 완료.

--- [50x] 데이터세트 결과 저장 중 ---
  -> 요약 결과 저장 완료: C:\FTO_data\FTO_training\data_20250827\50x_fp\50x_summary_evaluation.c

In [85]:
# 모델 Plot 생성 및 저장 (validation dataset)

print(f"Plot을 다음 기본 경로에 저장합니다: {OUTPATH}")

# PyCaret이 plot_type별로 저장하는 기본 파일 이름 정의
default_filenames = {
    'auc': 'AUC.png',
    'confusion_matrix': 'Confusion Matrix.png',
    'learning': 'Learning Curve.png',
    'feature': 'Feature Importance.png',
    'error': 'Prediction Error.png',
    'calibration': 'Calibration Curve.png'
}

data_suffixes = ['1x', '3x', '5x', '10x', '20x', '50x'] 

for suffix in data_suffixes:
    
    # Plot을 그리기 전에 해당 데이터로 setup을 다시 실행하여 세션을 활성화
    print(f"\n--- 데이터세트 [{suffix}]에 대한 PyCaret 세션 활성화 ---")
    x_train = globals()[f'x_train_{suffix}']
    y_train = globals()[f'y_train_{suffix}']
    df_train = pd.concat([x_train, y_train], axis=1)
    
    # 튜닝 때 사용했던 설정과 동일하게 setup을 실행
    setup(
        data=df_train,
        target='potency',
        session_id=42,
        train_size=0.9,
        fold=10,
        normalize=True,
        fix_imbalance=True,
        remove_outliers=True,
        n_jobs=-1, 
        verbose=False
    )
    
    # Plot 저장 폴더 생성
    plot_save_dir = os.path.join(OUTPATH, f"{suffix}_fp", "plots")
    os.makedirs(plot_save_dir, exist_ok=True)
    print(f"--- [{suffix}] 데이터세트 Plot 저장 시작 ---")

    # 튜닝된 모델들을 하나씩 순회
    for model_name, model_object in all_tuned_models[suffix].items():
        print(f"  -> 모델 [{model_name}]의 Plot 생성...")
        
        # 정의된 plot 종류를 순회
        for plot_type, default_name in default_filenames.items():
            try:
                # 1. 기본 이름으로 저장
                plot_model(model_object, plot=plot_type, save=True, verbose=False)
                
                # 2. 원하는 이름과 위치로 이동
                default_file_path = default_name
                final_filename = f"{suffix}_{model_name}_{plot_type}.png"
                final_save_path = os.path.join(plot_save_dir, final_filename)
                
                if os.path.exists(default_file_path):
                    shutil.move(default_file_path, final_save_path)
                else:
                    logging.warning(f"    - 기본 파일 '{default_file_path}'을 찾을 수 없습니다.")

            except Exception as e:
                logging.error(f"     - [{plot_type}] Plot 생성 실패: {e}")

    print(f"  ✅ [{suffix}] 데이터세트의 Plot 저장이 완료되었습니다.")

print("\n🎉 모든 Plot 파일 저장이 완료되었습니다!")

Plot을 다음 기본 경로에 저장합니다: C:\FTO_data\FTO_training\data_20250827

--- 데이터세트 [1x]에 대한 PyCaret 세션 활성화 ---
--- [1x] 데이터세트 Plot 저장 시작 ---
  -> 모델 [LogisticRegression]의 Plot 생성...
  -> 모델 [RandomForestClassifier]의 Plot 생성...
  -> 모델 [GradientBoostingClassifier]의 Plot 생성...
  -> 모델 [LGBMClassifier]의 Plot 생성...


2025-08-27 22:27:10,535 [ERROR] root:      - [auc] Plot 생성 실패: AUC plot not available for estimators with no predict_proba attribute.


  -> 모델 [RidgeClassifier]의 Plot 생성...


2025-08-27 22:27:20,097 [ERROR] root:      - [calibration] Plot 생성 실패: Calibration plot not available for estimators with no predict_proba attribute.


  -> 모델 [ExtraTreesClassifier]의 Plot 생성...
  ✅ [1x] 데이터세트의 Plot 저장이 완료되었습니다.

--- 데이터세트 [3x]에 대한 PyCaret 세션 활성화 ---
--- [3x] 데이터세트 Plot 저장 시작 ---
  -> 모델 [LogisticRegression]의 Plot 생성...
  -> 모델 [RandomForestClassifier]의 Plot 생성...
  -> 모델 [GradientBoostingClassifier]의 Plot 생성...
  -> 모델 [LGBMClassifier]의 Plot 생성...


...

  -> 모델 [ExtraTreesClassifier]의 Plot 생성...
  ✅ [20x] 데이터세트의 Plot 저장이 완료되었습니다.

--- 데이터세트 [50x]에 대한 PyCaret 세션 활성화 ---
--- [50x] 데이터세트 Plot 저장 시작 ---
  -> 모델 [LogisticRegression]의 Plot 생성...
  -> 모델 [RandomForestClassifier]의 Plot 생성...
  -> 모델 [GradientBoostingClassifier]의 Plot 생성...
  -> 모델 [LGBMClassifier]의 Plot 생성...


2025-08-27 22:49:54,740 [ERROR] root:      - [auc] Plot 생성 실패: AUC plot not available for estimators with no predict_proba attribute.


  -> 모델 [RidgeClassifier]의 Plot 생성...


2025-08-27 22:50:39,030 [ERROR] root:      - [calibration] Plot 생성 실패: Calibration plot not available for estimators with no predict_proba attribute.


  -> 모델 [ExtraTreesClassifier]의 Plot 생성...
  ✅ [50x] 데이터세트의 Plot 저장이 완료되었습니다.

🎉 모든 Plot 파일 저장이 완료되었습니다!


In [86]:
## 모델 평가 (evaluation)
import pandas as pd
from pycaret.classification import setup, evaluate_model

# 전체 데이터세트 목록
data_suffixes = ['1x', '3x', '5x', '10x', '20x', '50x'] 

# all_tuned_models 딕셔너리를 순회
for suffix in data_suffixes:
    # 해당 suffix에 대한 결과가 있는지 확인
    if suffix in all_tuned_models:
        
        # [추가] 평가 전에 해당 데이터로 setup을 다시 실행하여 세션을 활성화
        print(f"\n\n=================================================")
        print(f"--- 데이터세트 [{suffix}]에 대한 PyCaret 세션 활성화 ---")
        x_train = globals()[f'x_train_{suffix}']
        y_train = globals()[f'y_train_{suffix}']
        df_train = pd.concat([x_train, y_train], axis=1)
        
        # 튜닝 때 사용했던 설정과 동일하게 setup을 실행
        setup(
            data=df_train,
            target='potency',
            session_id=42,
            train_size=0.9,
            fold=10,
            normalize=True,
            fix_imbalance=True,
            remove_outliers=True,
            n_jobs=-1,
            verbose=False
        )
        
        print(f"--- 데이터세트 [{suffix}] 모델 평가 시작 ---")
        
        # 튜닝된 모델들을 하나씩 꺼내어 평가
        for model_name, model_object in all_tuned_models[suffix].items():
            print(f"\n>>> 모델 [{model_name}]의 평가 대시보드를 생성합니다...")
            
            # evaluate_model 실행
            evaluate_model(model_object)
    else:
        print(f"\n--- 데이터세트 [{suffix}]에 대한 튜닝 결과가 없습니다. 건너뜁니다. ---")

print("\n\n🎉 모든 모델 평가가 완료되었습니다!")



--- 데이터세트 [1x]에 대한 PyCaret 세션 활성화 ---
--- 데이터세트 [1x] 모델 평가 시작 ---

>>> 모델 [LogisticRegression]의 평가 대시보드를 생성합니다...


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…


>>> 모델 [RandomForestClassifier]의 평가 대시보드를 생성합니다...


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…


>>> 모델 [GradientBoostingClassifier]의 평가 대시보드를 생성합니다...


...


>>> 모델 [RidgeClassifier]의 평가 대시보드를 생성합니다...


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…


>>> 모델 [ExtraTreesClassifier]의 평가 대시보드를 생성합니다...


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…



🎉 모든 모델 평가가 완료되었습니다!


In [88]:
## 모델 블렌딩

# 블렌딩된 모델을 저장할 딕셔너리
all_blended_models = {}

# 블렌딩 결과를 저장할 딕셔너리
all_blended_results = {}

# 블렌딩할 데이터세트 목록
data_suffixes = ['1x', '3x', '5x', '10x', '20x', '50x']

for suffix in data_suffixes:
    print(f"\n--- 데이터세트 [{suffix}] 모델 블렌딩 시작 ---")
    
    # 1. [핵심] 해당 데이터세트의 튜닝된 모델들을 F1 점수 기준으로 정렬
    summary_df = all_summary_dfs[suffix]
    mean_scores = summary_df[summary_df['Type'] == 'Mean'].copy()
    top_models_df = mean_scores.sort_values(by='F1', ascending=False)
    
    # 2. 상위 4개 모델의 이름(클래스명)을 리스트로 추출
    top_3_model_names = top_models_df['Model'].head(4).tolist()
    
    # 3. 이름에 해당하는 실제 모델 객체를 가져오기
    models_to_blend = [all_tuned_models[suffix][name] for name in top_3_model_names]
    
    print(f" -> 블렌딩 대상 모델: {top_3_model_names}")
    
    # 4. 모델 블렌딩 실행
    # method='soft'는 각 모델의 예측 '확률'을 평균내어 더 부드러운 결정을 내림
    blended_model = blend_models(
        estimator_list=models_to_blend, 
        optimize='F1',
        method='soft', 
        verbose=False
    )
    
    # 5. 블렌딩 결과(성능 점수) 저장
    blended_results_df = pull().copy()
    
    # 6. 각 딕셔너리에 결과 저장
    all_blended_models[suffix] = blended_model
    all_blended_results[suffix] = blended_results_df
    
    print(" -> 블렌딩 후 성능:")
    display(blended_results_df)

print("\n🎉 모든 데이터세트의 모델 블렌딩이 완료되었습니다!")

2025-08-27 23:20:04,785 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.



--- 데이터세트 [1x] 모델 블렌딩 시작 ---
 -> 블렌딩 대상 모델: ['GradientBoostingClassifier', 'LGBMClassifier', 'RandomForestClassifier', 'LogisticRegression']


2025-08-27 23:20:04,787 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


 -> 블렌딩 후 성능:


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9959,0.9950,0.8421,0.9412,0.8889,0.8868,0.8882
1,0.9928,0.9924,0.7895,0.8333,0.8108,0.8071,0.8074
2,0.9969,0.9981,0.9474,0.9000,0.9231,0.9215,0.9218
3,0.9918,0.9960,0.6316,0.9231,0.7500,0.7460,0.7599
4,0.9959,0.9782,0.8947,0.8947,0.8947,0.8926,0.8926
5,0.9928,0.9920,0.7895,0.8333,0.8108,0.8071,0.8074
6,0.9876,0.9896,0.5789,0.7333,0.6471,0.6409,0.6455
7,0.9969,0.9991,0.8947,0.9444,0.9189,0.9173,0.9177
8,0.9907,0.9883,0.8947,0.7083,0.7907,0.7860,0.7916


...


--- 데이터세트 [50x] 모델 블렌딩 시작 ---
 -> 블렌딩 대상 모델: ['LGBMClassifier', 'LogisticRegression', 'GradientBoostingClassifier', 'ExtraTreesClassifier']


2025-08-27 23:31:29,908 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.


 -> 블렌딩 후 성능:


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9938,0.9953,0.8421,0.8421,0.8421,0.8390,0.8390
1,0.9928,0.9811,0.7895,0.8333,0.8108,0.8071,0.8074
2,0.9969,0.9959,0.9474,0.9000,0.9231,0.9215,0.9218
3,0.9918,0.9659,0.6316,0.9231,0.7500,0.7460,0.7599
4,0.9959,0.9828,0.8947,0.8947,0.8947,0.8926,0.8926
5,0.9928,0.9893,0.8421,0.8000,0.8205,0.8168,0.8171
6,0.9876,0.9912,0.5789,0.7333,0.6471,0.6409,0.6455
7,0.9969,0.9988,0.8947,0.9444,0.9189,0.9173,0.9177
8,0.9907,0.9871,0.8947,0.7083,0.7907,0.7860,0.7916



🎉 모든 데이터세트의 모델 블렌딩이 완료되었습니다!


In [89]:
## 전체 데이터세트 대상, 블렌딩 모델 최종 성능 평가
import pandas as pd

# 1. 전체 데이터세트 목록 정의
data_suffixes = ['1x', '3x', '5x', '10x', '20x', '50x'] 

# 최종 예측 결과와 성능 지표를 저장할 딕셔너리
all_final_predictions = {}
all_final_metrics = {}

print(f"\n=============== 전체 데이터세트 최종 성능 평가 시작 ===============")

# 2. for 반복문을 사용하여 모든 데이터세트를 순회
for suffix in data_suffixes:
    print(f"\n--- [{suffix}] 데이터세트 평가 중 ---")
    
    # 해당 suffix에 대한 블렌딩 모델이 있는지 확인
    if suffix in all_blended_models:
        # 3. 해당 데이터세트의 블렌딩 모델 및 테스트 데이터 준비
        blended_model = all_blended_models[suffix]
        x_test = globals()[f'x_test_{suffix}']
        y_test = globals()[f'y_test_{suffix}']
        test_df = pd.concat([x_test, y_test], axis=1)

        # 4. predict_model을 사용하여 최종 성능 평가
        final_predictions = predict_model(blended_model, data=test_df)
        
        # 5. 결과 저장
        all_final_predictions[suffix] = final_predictions
        all_final_metrics[suffix] = pull() # predict_model 직후의 성능표 저장
        
    else:
        print(f" -> [{suffix}]에 대한 블렌딩 모델이 없어 건너뜁니다.")

print("\n\n🎉 모든 데이터세트에 대한 최종 평가가 완료되었습니다!")

# 6. 전체 결과 요약 출력
print("\n\n=============== 최종 성능 요약 ===============")
for suffix, metrics_df in all_final_metrics.items():
    print(f"\n--- [{suffix}] Blended Model Final Score ---")
    display(metrics_df)

2025-08-27 23:34:00,692 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.



=============== 전체 데이터세트 최종 성능 평가 시작 ===============

--- [1x] 데이터세트 평가 중 ---


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9362,0.9801,0.8696,1.0000,0.9302,0.8719,0.8792


2025-08-27 23:34:02,601 [DEBUG] matplotlib.pyplot: Loaded backend module://matplotlib_inline.backend_inline version unknown.



--- [3x] 데이터세트 평가 중 ---


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9884,0.9844,0.875,1.0,0.9333,0.927,0.9295



--- [20x] Blended Model Final Score ---


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.996,0.9995,1.0,0.9231,0.96,0.9579,0.9587



--- [50x] Blended Model Final Score ---


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.995,0.9843,0.7917,0.95,0.8636,0.8611,0.8648


임시코드

In [ ]:
## 모델 블랜딩

# 1. 블렌딩할 모델들이 있는 데이터세트 선택
target_suffix = '1x'

# 2. 해당 데이터세트의 튜닝된 모델 객체들을 리스트로 만들기
# all_tuned_models 딕셔너리에서 모델 객체(value)들만 가져옵니다.
models_to_blend = list(all_tuned_models[target_suffix].values())

print(f"[{target_suffix}] 데이터세트의 다음 모델들을 블렌딩합니다:")
for model in models_to_blend:
    print(f" - {str(model).split('(')[0]}")

# 3. 모델 블렌딩 실행
# fold=5는 교차 검증을 5번 수행한다는 의미입니다.
blended_model = blend_models(estimator_list=models_to_blend, 
                             fold=5,
                             optimize='F1')

# 4. 블렌딩된 모델의 성능 확인
print("\n--- 블렌딩된 모델의 성능 ---")
blended_results = pull()
display(blended_results)

In [ ]:
## 블랜딩 모델 성능평가

# 1. 블렌딩 모델을 만들 때 사용했던 데이터세트 지정
# 이전 단계에서 '1x'를 사용했으므로 동일하게 설정합니다.
target_suffix = '1x' 

print(f"\n=============== 블렌딩 모델 최종 성능 평가 시작 ({target_suffix}) ===============")

# 2. 해당 데이터세트의 테스트 데이터 준비
x_test = globals()[f'x_test_{target_suffix}']
y_test = globals()[f'y_test_{target_suffix}']
test_df = pd.concat([x_test, y_test], axis=1)

# 3. predict_model을 사용하여 블렌딩 모델의 최종 성능 평가
blended_model_predictions = predict_model(blended_model, data=test_df)

print("\n🎉 블렌딩 모델의 최종 평가가 완료되었습니다!")